## Import and Load data file

In [ ]:
!pip install nltk
!pip install tensorflow
!pip install keras


import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

## Pre-process data

Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [190]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

lemmatize each word and remove duplicate words from the list

In [191]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

63 documents
14 classes ['adverse_drug', 'baggage_allowance', 'baggage_weight', 'blood_pressure', 'blood_pressure_search', 'flight_booking', 'flight_status', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'set_reminder', 'thanks']
116 unique lemmatized words ["'s", ',', '7.00pm', 'a', 'adverse', 'alarm', 'all', 'allowance', 'an', 'anyone', 'are', 'at', 'awesome', 'baggage', 'be', 'behavior', 'blood', 'book', 'bring', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'date', 'day', 'delay', 'delayed', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'flight', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'if', 'is', 'later', 'limit', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'luggage', 'management', 'me', 'module', 'much', 'my', 'nearby', 'next', 'nice', 'of', 'offered', 'on', 'open', 'patient', 'pharmacy', 'policy', 'pressure', 'provid

## Create training and testing data

In [192]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\abhil\AppData\Local\Temp\ipykernel_14260\39905841.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


## Create Model

In [193]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
13/13 [==============================] - 0s 2ms/step - loss: 2.6943 - accuracy: 0.0635
Epoch 2/200
13/13 [==============================] - 0s 2ms/step - loss: 2.6500 - accuracy: 0.0635
Epoch 3/200
13/13 [==============================] - 0s 2ms/step - loss: 2.5795 - accuracy: 0.0952
Epoch 4/200
13/13 [==============================] - 0s 2ms/step - loss: 2.4922 - accuracy: 0.1905
Epoch 5/200
13/13 [==============================] - 0s 2ms/step - loss: 2.4566 - accuracy: 0.2222
Epoch 6/200
13/13 [==============================] - 0s 2ms/step - loss: 2.4144 - accuracy: 0.2698
Epoch 7/200
13/13 [==============================] - 0s 2ms/step - loss: 2.2638 - accuracy: 0.4921
Epoch 8/200
13/13 [==============================] - 0s 2ms/step - loss: 2.2512 - accuracy: 0.3016
Epoch 9/200
13/13 [==============================] - 0s 2ms/step - loss: 2.0029 - accuracy: 0.4921
Epoch 10/200
13/13 [==============================] - 0s 1ms/step - loss: 1.9185 - accuracy: 0.5079
Epoch 11/

Epoch 83/200
13/13 [==============================] - 0s 1ms/step - loss: 0.1333 - accuracy: 0.9365
Epoch 84/200
13/13 [==============================] - 0s 1ms/step - loss: 0.0798 - accuracy: 0.9683
Epoch 85/200
13/13 [==============================] - 0s 1ms/step - loss: 0.0856 - accuracy: 0.9841
Epoch 86/200
13/13 [==============================] - 0s 2ms/step - loss: 0.1630 - accuracy: 0.9524
Epoch 87/200
13/13 [==============================] - 0s 1ms/step - loss: 0.0623 - accuracy: 0.9841
Epoch 88/200
13/13 [==============================] - 0s 1ms/step - loss: 0.1493 - accuracy: 0.9524
Epoch 89/200
13/13 [==============================] - 0s 1ms/step - loss: 0.1174 - accuracy: 0.9365
Epoch 90/200
13/13 [==============================] - 0s 1ms/step - loss: 0.1662 - accuracy: 0.9524
Epoch 91/200
13/13 [==============================] - 0s 1ms/step - loss: 0.0996 - accuracy: 0.9683
Epoch 92/200
13/13 [==============================] - 0s 1ms/step - loss: 0.0425 - accuracy: 1.0000


In [194]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abhil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abhil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\abhil\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [195]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [196]:
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary m
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [197]:
def predict_class(sentence, model):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [198]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [199]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

## GUI Interface

In [200]:
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0", END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("My Personal Assistant")
base.geometry("600x500")
base.resizable(width=False, height=False)

ChatLog = Text(base, bd=0, bg="white", height="8", width="75", font="Arial")
ChatLog.config(state=DISABLED)

scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart", width=10)
ChatLog['yscrollcommand'] = scrollbar.set

SendButton = Button(base, font=("Verdana", 12, 'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#0066cc", activebackground="#3399ff", fg='#ffffff',
                    command=send)
EntryBox = Text(base, bd=0, bg="white", width="50", height="5", font="Arial", background="#dddddd")

scrollbar.place(x=575, y=6, height=386)
ChatLog.place(x=6, y=6, height=386, width=565)
EntryBox.place(x=150, y=401, height=40, width=440)
SendButton.place(x=6, y=401, height=40)

In [ ]:
base.mainloop()

1/1 [==============================] - 0s 19ms/step
